In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, text
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
#  Db connection
engine = create_engine("sqlite:///fintrack.db")
Base = declarative_base()
#session for db interaction
Session = sessionmaker(bind=engine)
session = Session()

In [2]:
class Category(Base):           # category table for expense
    __tablename__ = "categories"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    expenses = relationship("Expense", back_populates="category")
#expense table
class Expense(Base):
    __tablename__ = "expenses"
    id = Column(Integer, primary_key=True)
    title = Column(String)
    amount = Column(Integer)
    date = Column(String)
    category_id = Column(Integer, ForeignKey("categories.id"))
    category = relationship("Category", back_populates="expenses")

class Subscription(Base):
    __tablename__ = "subscriptions"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    amount = Column(Integer)
    next_date = Column(String)
    #budget table for montly limits
class Budget(Base):      
    __tablename__ = "budgets"
    id = Column(Integer, primary_key=True)
    month = Column(String)
    limit = Column(Integer)
#connect to database using engine
Base.metadata.create_all(engine)
#func for new expensecategory 
def add_category():
    name = input("Category name: ")
    session.add(Category(name=name))
    session.commit()
    print("Category added")

#adding new record
def add_expense():
    title = input("Expense title: ")
    amount = int(input("Amount: "))
    date = input("Date (YYYY-MM-DD): ")
    category_id = int(input("Category ID: "))
    session.add(Expense(title=title,amount=amount,date=date,category_id=category_id))
    session.commit()
    print("expense added")

def update_expense():
    eid = int(input("expense id: "))
    expense = session.query(Expense).filter(Expense.id == eid).first()
    if expense:
        expense.amount = int(input("new amount: "))
        session.commit()
        print("Expense updated")
    else:
        print("Expense not found")
#expense delete 
def delete_expense():
    eid = int(input("Expense id: "))
    expense = session.query(Expense).filter(Expense.id == eid).first()
    if expense:
        session.delete(expense)
        session.commit()
        print("expense deleted")
    else:
        print("Expense not found")
def search_by_date():
    date = input("Enter date: ")
    expenses = session.query(Expense).filter(Expense.date == date).all()
    for e in expenses:
        print(e.title, e.amount, e.date)

#Sql report
def category_report():
    sql = """SELECT categories.name, SUM(expenses.amount) From categories JOIN expenses ON 
    categories.id = expenses.category_id GROUP BY categories.name"""
    result = session.execute(text(sql))
    print("Category Wise Expense Report")
    for row in result:
        print(row[0], ",", row[1])
#budeget limit function
def set_budget():
    month = input("Month: ")
    limit = int(input("Month limit: "))
    session.add(Budget(month=month, limit=limit))
    session.commit()
    print("budget set")
def budget_alert():
    month = input("Month: ")
    total = session.execute(text("SELECT SUM(amount) FROM expenses WHERE date LIKE :m"),{"m": f"{month}%"}).scalar()
    budget = session.query(Budget).filter(Budget.month == month).first()
    if budget and total and total > budget.limit:
        print("Budget exceeded")
    else:
        print("spending within limit")

In [ ]:
# CLI part  it hels in the application of different functions

while True:
    print("""------FINTRACK PRO --------
1. Add Category
2. Add Expense
3. Update Expense
4.Delete Expense
5. Search Expense by Date
6.Category Expense Report
7. Set Monthly budget
8. Budget alert
9. Exit""")
    choice = input("Choose: ")
    if choice == "1":
        add_category()
    elif choice == "2":
        add_expense()
    elif choice == "3":
        update_expense()
    elif choice == "4":
        delete_expense()
    elif choice == "5":
        search_by_date()
    elif choice == "6":
        category_report()
    elif choice == "7":
        set_budget()
    elif choice == "8":
        budget_alert()
    elif choice == "9":
        break
    else:
        print("invalid choice")

------FINTRACK PRO --------
1. Add Category
2. Add Expense
3. Update Expense
4.Delete Expense
5. Search Expense by Date
6.Category Expense Report
7. Set Monthly budget
8. Budget alert
9. Exit
